In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from google.colab import drive

In [2]:
# Daftar sapaan dan respons yang sesuai
greetings = {
    'hai': 'Hai juga!',
    'hello': 'Hello juga!',
    'halo': 'Halo juga!',
    'hi': 'Hi juga!',
    'good day': 'Good day juga!',
    'selamat pagi': 'Selamat pagi juga!',
    'selamat siang': 'Selamat siang juga!',
    'selamat sore': 'Selamat sore juga!',
    'selamat malam': 'Selamat malam juga!'
}

In [3]:
# Hubungkan Colab ke Google Drive
drive.mount('/content/drive')


# Ganti 'DatasetChatbot.xlsx' dengan path file Anda di Google Drive
file_path = '/content/drive/My Drive/NLP_SZ/Projek_UAS/DatasetChatbot.xlsx'

# Load data
data = pd.read_excel(file_path, skiprows=1 )
data.head()

Mounted at /content/drive


,Question_ID,Questions,Answers
0,Q001,Apa fungsi dari rangka tubuh manusia?,"Menegakkan tubuh, Memberi bentuk tubuh, Memban..."
1,Q002,Apa itu rangka?,Rangka adalah ratusan tulang yang saling berhu...
2,Q003,Mengapa tubuh kita bisa berdiri tegak?,Tubuh kita bisa berdiri tegak karena adanya ra...
3,Q004,Tulang apa saja yang membentuk lengan?,"Tulang lengan atas, tulang hasta, tulang pengu..."
4,Q005,Tulang apa saja yang membentuk rongga dada?,"Tulang rusuk sejati, tulang rusuk palsu, tulan..."


In [4]:
# Drop baris kosong
data.dropna(subset=['Questions', 'Answers'], inplace=True)

# Melihat distribusi data
print(data.info())

# Menyederhanakan pertanyaan menjadi huruf kecil
data['Questions'] = data['Questions'].str.lower()


<class 'pandas.core.frame.DataFrame'>
Index: 219 entries, 0 to 219
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Question_ID  219 non-null    object
 1   Questions    219 non-null    object
 2   Answers      219 non-null    object
dtypes: object(3)
memory usage: 6.8+ KB
None


In [5]:
# Memisahkan fitur dan label
X = data['Questions']
y = data['Answers']

# Membagi dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:
# TF-IDF Vectorizer dengan pembatasan fitur
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [7]:
# Inisialisasi Naive Bayes
model = MultinomialNB(alpha=0)
model.fit(X_train_tfidf, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/naive_bayes.py:897: RuntimeWarning: divide by zero encountered in log
  self.feature_log_prob_ = np.log(smoothed_fc) - np.log(


MultinomialNB(alpha=0)

In [8]:
# Validasi pada data
y_train_pred = model.predict(X_train_tfidf)
print(f'Akurasi data : {accuracy_score(y_train, y_train_pred) * 100:.2f}%')


Akurasi data : 99.43%


In [9]:
import joblib

# Menyimpan model dan vectorizer
joblib.dump(model, '/content/drive/My Drive/NLP_SZ/Projek_UAS/model.pkl')
joblib.dump(vectorizer, '/content/drive/My Drive/NLP_SZ/Projek_UAS/vektor.pkl')

['/content/drive/My Drive/NLP_SZ/Projek_UAS/vektor.pkl']

In [10]:
print("Halo! Saya adalah chatbot. Anda dapat bertanya apa saja. Ketik 'keluar' untuk keluar.")

while True:
    user_input = input("Anda: ").lower()

    if user_input == 'keluar':
        print("Chatbot: Terima kasih! Sampai jumpa.")
        break
    elif user_input in greetings:
        # Jawaban berdasarkan sapaan
        print(f"Chatbot: {greetings[user_input]}")
    else:
        # Transformasi input pengguna
        user_input_tfidf = vectorizer.transform([user_input])
        # Prediksi jawaban
        answer = model.predict(user_input_tfidf)
        print(f"Chatbot: {answer[0]}")


Halo! Saya adalah chatbot. Anda dapat bertanya apa saja. Ketik 'keluar' untuk keluar.
Anda: halo
Chatbot: Halo juga!
Anda: apa itu rangka
Chatbot: Rangka adalah ratusan tulang yang saling berhubungan di dalam tubuh. Rangka tubuh manusia terdiri atas tengkorak, rangka penyusun badan, dan rangka penyusun anggota gerak
Anda: simbiosis
Chatbot: Simbiosis adalah Hubungan khas antara dua makhluk hidup yang hidup bersama-sama.
Anda: keluar
Chatbot: Terima kasih! Sampai jumpa.
